# Breweries By Zip Code In Colorado Springs, CO

1. Introduction/Business Problem

Craft beer is very popular in Colorado and more breweries are looking for places to expand. Colorado Springs is the second larges metro area in Colorado with a growing population. A brewer from Denver would like to open a second location and thinks Colorado Springs is an up an coming location. What brewers have found is that they are more successfull if there is more than one brewery nearby. However, if there are too many breweries in one area they can oversaturate the market.

Prospective brewers and restaurantuers would like to see where breweries exist and their average rating to help decide where to open their next location.

2. Data Sources

In addition to the Four Square Data for Colorado Springs existing breweries, I will scrape the following website for zip code data to Colorado Springs.

https://www.zipcodestogo.com/Colorado/

For Lat Long Data by Zip Code I will scrape this website:

https://gist.github.com/abatko/ee7b24db82a6f50cfce02afafa1dfd1e

Overlaying the foursquare ratings and number of existing breweries on a map will help prospective breweres decide where they might open their next location and if they have an opportunity to get a higher rating.


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Scrape website for all Colorado Zip Codes

In [2]:
co = []
df = pd.read_html('https://www.zipcodestogo.com/Colorado/')[1]
co = df.values.tolist()
co

[['Zip Codes for the State of  Colorado',
  'Zip Codes for the State of  Colorado',
  'Zip Codes for the State of  Colorado',
  'Zip Codes for the State of  Colorado'],
 ['Zip Code', 'City', 'County', 'Zip Code Map'],
 ['80001', 'Arvada', 'Jefferson', 'View Map'],
 ['80002', 'Arvada', 'Jefferson', 'View Map'],
 ['80003', 'Arvada', 'Jefferson', 'View Map'],
 ['80004', 'Arvada', 'Jefferson', 'View Map'],
 ['80005', 'Arvada', 'Jefferson', 'View Map'],
 ['80006', 'Arvada', 'Jefferson', 'View Map'],
 ['80007', 'Arvada', 'Jefferson', 'View Map'],
 ['80010', 'Aurora', 'Arapahoe', 'View Map'],
 ['80011', 'Aurora', 'Adams', 'View Map'],
 ['80012', 'Aurora', 'Arapahoe', 'View Map'],
 ['80013', 'Aurora', 'Arapahoe', 'View Map'],
 ['80014', 'Aurora', 'Arapahoe', 'View Map'],
 ['80015', 'Aurora', 'Arapahoe', 'View Map'],
 ['80016', 'Aurora', 'Arapahoe', 'View Map'],
 ['80017', 'Aurora', 'Arapahoe', 'View Map'],
 ['80018', 'Aurora', 'Arapahoe', 'View Map'],
 ['80019', 'Aurora', 'Adams', 'View Map'],

### Create list of only Colorado Springs zip codes and create dataframe with that data

In [4]:
csco=[]
for entry in co:
    if entry[1] == 'Colorado Springs':
        csco.append(entry)


In [5]:
cs = pd.DataFrame(csco)
cs.rename(columns={0:'Zip'}, inplace=True)
cs.rename(columns={1:'City'}, inplace=True)
cs.rename(columns={2:'County'}, inplace=True)
del cs[3]
cs.head()

,Zip,City,County
0,80901,Colorado Springs,El Paso
1,80903,Colorado Springs,El Paso
2,80904,Colorado Springs,El Paso
3,80905,Colorado Springs,El Paso
4,80906,Colorado Springs,El Paso


### Scrape website for Latitude and Longitude data on US zip codes.

In [6]:
zip_lat_long = []
dfzip = pd.read_html('https://gist.github.com/abatko/ee7b24db82a6f50cfce02afafa1dfd1e')[0]
zip_lat_long = dfzip.values.tolist()

In [7]:
zip_lat_long[32913]

[nan, '9950761.147656-149.753642']

In [8]:
zipll=[]
for i in zip_lat_long:
    zipll.append(i[1])
    
zipll[32913]

'9950761.147656-149.753642'

### The data came in as comma seperated so creating a dataframe by spliting on comma. Also upon looking at some list values it became apparent that when the data was scrapped that some of the commas were overlooked and created datapoints like the one above so we added a column to check the lenth of the data in first column.

In [9]:
zip= pd.DataFrame([sub.split(",") for sub in zipll])
zip.columns = zip.iloc[0]
zip = zip.drop(zip.index[0])
zip.rename(columns={"ZIP":'Zip'}, inplace=True)
zip['Zip_Len']= zip['Zip'].str.len()
zip.head()

,Zip,LAT,LNG,Zip_Len
1,00601,18.180555,-66.749961,5
2,00602,18.361945,-67.175597,5
3,00603,18.455183,-67.119887,5
4,00606,18.158327,-66.932928,5
5,00610,18.295366,-67.125135,5


### created a new data frame for the data that did not come across with commas due do some additional data preperation. Also created a final dataframe "zipdone" for the data that was comma seperated.

In [10]:
zipwork = zip[zip['Zip_Len']>5].copy()
zipdone = [x for x in zip if x not in zipwork]
zipdone = pd.DataFrame(zipdone)
zipwork = pd.DataFrame(zipwork)
zipwork.head()

,Zip,LAT,LNG,Zip_Len
19508,5742045.292760-100.149046,None,None,25
19527,5744245.061750-100.158139,None,None,25
19548,5747245.435717-100.068105,None,None,25
19556,5750144.527600-100.309037,None,None,25
19557,5752044.852798-100.193658,None,None,25


### Split the data using string functions.

In [11]:
zipwork['z1']=zipwork['Zip'].str[0:5]
zipwork['lat1']=zipwork['Zip'].apply(lambda st: st[5:st.find("-")])
zipwork['lng2']=zipwork['Zip'].apply(lambda st: st[st.find("-"):])
zipwork['Zip']=zipwork['z1']
zipwork['LAT']=zipwork['lat1']
zipwork['LNG']=zipwork['lng2']
del zipwork['z1']
del zipwork['lat1']
del zipwork['lng2']
zipwork.head()

,Zip,LAT,LNG,Zip_Len
19508,57420,45.292760,-100.149046,25
19527,57442,45.061750,-100.158139,25
19548,57472,45.435717,-100.068105,25
19556,57501,44.527600,-100.309037,25
19557,57520,44.852798,-100.193658,25


### Combine the dataframes back together so that all zip codes are in the same dataframe.

In [12]:
zipfinal = pd.concat([zipdone, zipwork])
zipfinal.tail()

,Zip,LAT,LNG,Zip_Len
32909,99503,62.043951,-158.175667,25
32910,99504,61.204600,-149.746095,25
32911,99505,61.256994,-149.601615,25
32912,99506,61.253462,-149.810741,25
32913,99507,61.147656,-149.753642,25


### Merge the zip Lat Long data with the Colorado Springs zip codes and drop the zip codes without Lat, Long data because with some additional research, these zipcodes are not actively used yet.

In [13]:
csprings = pd.merge(cs,zipfinal,on = "Zip", how='inner')
csprings.head()

,Zip,City,County,LAT,LNG,Zip_Len
0,80903,Colorado Springs,El Paso,38.831526,-104.815062,25
1,80904,Colorado Springs,El Paso,38.860829,-104.876163,25
2,80905,Colorado Springs,El Paso,38.818910,-104.838348,25
3,80906,Colorado Springs,El Paso,38.752594,-104.878211,25
4,80907,Colorado Springs,El Paso,38.878231,-104.827015,25


In [14]:
csz = csprings.dropna()
CS = pd.DataFrame(csz)
CS['LAT'] = CS['LAT'].astype(float)
CS['LNG'] = CS['LNG'].astype(float)
CS

,Zip,City,County,LAT,LNG,Zip_Len
0,80903,Colorado Springs,El Paso,38.831526,-104.815062,25
1,80904,Colorado Springs,El Paso,38.860829,-104.876163,25
2,80905,Colorado Springs,El Paso,38.818910,-104.838348,25
3,80906,Colorado Springs,El Paso,38.752594,-104.878211,25
4,80907,Colorado Springs,El Paso,38.878231,-104.827015,25
5,80908,Colorado Springs,El Paso,39.047585,-104.690540,25
6,80909,Colorado Springs,El Paso,38.853475,-104.775218,25
7,80910,Colorado Springs,El Paso,38.812066,-104.774580,25
8,80911,Colorado Springs,El Paso,38.752916,-104.723477,25
9,80913,Colorado Springs,El Paso,38.687462,-104.748767,25


### Create a map of all Colorado Springs Zip Codes.

In [15]:
address = 'Colorado Springs, CO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Colorado Springs are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Colorado Springs are 38.8339578, -104.8253485.


In [218]:
# create map of Colorado Springs using latitude and longitude values
map_csprings = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for i in range(len(CS)):
    label = '{}'.format(CS['Zip'][i])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [CS['LAT'][i], CS['LNG'][i]],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_csprings)
    #folium.Marker(
        #[CS['LAT'][i], CS['LNG'][i]],
        #popup=label,
        #icon=folium.Icon(
            #icon_color='#FFD700',
            #icon='beer',
            #prefix='fa'),
        #).add_to(map_csprings)
    
map_csprings

### Put together FourSquare credentials.

In [30]:
CLIENT_ID = 'H5QTVL2PZDXPRTBG3O4BCJQ5BMOHZ4LCAJUKWDHWCSZBW4PA' # your Foursquare ID
CLIENT_SECRET = 'GDAFWVSE4MRTVEM0FDTYGY2MU2KJMYO5BDY2FXVK3Y5GCYN2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: H5QTVL2PZDXPRTBG3O4BCJQ5BMOHZ4LCAJUKWDHWCSZBW4PA
CLIENT_SECRET:GDAFWVSE4MRTVEM0FDTYGY2MU2KJMYO5BDY2FXVK3Y5GCYN2


In [18]:
x = 6
CS.loc[x, 'Zip']

'80909'

In [19]:
CS_latitude = CS.loc[x, 'LAT']
CS_longitude = CS.loc[x, 'LNG']

CS_zip = CS.loc[x, 'Zip']

print('Latitude and longitude values of {} are {}, {}.'.format(CS_zip, 
                                                               CS_latitude, 
                                                               CS_longitude))

Latitude and longitude values of 80909 are 38.853475, -104.775218.


### Create Query of Foursquare for only Breweriews within 150K of Colorado Springs. I used a really large radius just to get more data.

In [20]:
LIMIT = 1000
Query = 'Brewery'
radius = 150000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&query={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    CS_latitude, 
    CS_longitude, 
    radius, 
    Query)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=H5QTVL2PZDXPRTBG3O4BCJQ5BMOHZ4LCAJUKWDHWCSZBW4PA&client_secret=GDAFWVSE4MRTVEM0FDTYGY2MU2KJMYO5BDY2FXVK3Y5GCYN2&v=20180605&ll=38.853475,-104.775218&radius=150000&query=Brewery'

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6120f6d9971a8e123357cc59'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Colorado Springs',
  'headerFullLocation': 'Colorado Springs',
  'headerLocationGranularity': 'city',
  'query': 'brewery',
  'totalResults': 271,
  'suggestedBounds': {'ne': {'lat': 40.20347635000135,
    'lng': -103.04491188622788},
   'sw': {'lat': 37.503473649998654, 'lng': -106.50552411377211}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '512cfb53e4b01674115e19b8',
       'name': 'Red Leg Brewing Company',
       'location': {'address': '4630 Forge Rd',
        'crossStreet': 'Garden of the Gods',
        'lat': 38.898057,

In [22]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.location.postalCode', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.id']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,postalCode,categories,lat,lng,id
0,Red Leg Brewing Company,80907,Brewery,38.898057,-104.841728,512cfb53e4b01674115e19b8
1,Cerberus Brewing Company,80905,Brewery,38.833003,-104.837443,57c878a1498e9ad24cf1158d
2,Storybook Brewing,80907,Brewery,38.877701,-104.812470,54750b26498e21a5eda35f5d
3,Phantom Canyon Brewing Company,80903,Brewery,38.834215,-104.825177,4ab97703f964a5207e7f20e3
4,Bristol Brewing Company,80905,Brewery,38.811236,-104.827392,4af49ef4f964a5206af421e3
5,Trinity Brewing Company,80907,Brewery,38.897386,-104.854334,4b118eadf964a5203c7f23e3
6,Manitou Brewing Company,80829,Brewery,38.857072,-104.915789,53138f74498e349943bc3cde
7,Pikes Peak Brewing Company,80132,Brewery,39.095454,-104.860264,4d8e5cf31716a14364e730f7
8,Nano 108 Brewing Co,80915,Brewery,38.866652,-104.717830,5274485711d283dea3d0fa0a
9,Smiling Toad Brewing,80905,Brewery,38.806723,-104.838538,50e374bce4b0a6d1deb767d4


### Below I created a list of the venue ID so that I can do a premium API call on each venue to get the rating of each brewery. I used this list to add to a list of url for each premium call, then iterated through that list to create another list of the response.

In [24]:
VENUE_ID = []
for i in nearby_venues['id']:
    VENUE_ID.append(i)


In [25]:
venurls = []

for i in VENUE_ID:
    venurls.append('https://api.foursquare.com/v2/venues/'+i+'?&client_id='+CLIENT_ID+'&client_secret='+CLIENT_SECRET+'&v='+VERSION)


In [26]:
rating = []
for i in venurls:
    rating.append(requests.get(i).json())

rating

[{'meta': {'code': 200, 'requestId': '6120f70e8bc08c1c1cfb598f'},
  'response': {'venue': {'id': '512cfb53e4b01674115e19b8',
    'name': 'Red Leg Brewing Company',
    'contact': {'phone': '7195983776',
     'formattedPhone': '(719) 598-3776',
     'twitter': 'redlegbrewco',
     'facebook': '290972717668895',
     'facebookUsername': 'redlegbrewco',
     'facebookName': 'Red Leg Brewing Company'},
    'location': {'address': '4630 Forge Rd',
     'crossStreet': 'Garden of the Gods',
     'lat': 38.898057,
     'lng': -104.841728,
     'labeledLatLngs': [{'label': 'entrance',
       'lat': 38.898056,
       'lng': -104.841537},
      {'label': 'display', 'lat': 38.898057, 'lng': -104.841728}],
     'postalCode': '80907',
     'cc': 'US',
     'city': 'Colorado Springs',
     'state': 'CO',
     'country': 'United States',
     'formattedAddress': ['4630 Forge Rd (Garden of the Gods)',
      'Colorado Springs, CO 80907',
      'United States']},
    'canonicalUrl': 'https://foursquare.c

### Create data frame with API data on rating.

In [101]:
score = pd.DataFrame(json_normalize(rating))
score.sort_values('response.venue.location.city')
score.drop(score[score['response.venue.location.city']!='Colorado Springs'].index, inplace=True)
score.head()

,meta.code,meta.requestId,response.venue.id,response.venue.name,response.venue.contact.phone,response.venue.contact.formattedPhone,response.venue.contact.twitter,response.venue.contact.facebook,response.venue.contact.facebookUsername,response.venue.contact.facebookName,response.venue.location.address,response.venue.location.crossStreet,response.venue.location.lat,response.venue.location.lng,response.venue.location.labeledLatLngs,response.venue.location.postalCode,response.venue.location.cc,response.venue.location.city,response.venue.location.state,response.venue.location.country,response.venue.location.formattedAddress,response.venue.canonicalUrl,response.venue.categories,response.venue.verified,response.venue.stats.tipCount,response.venue.url,response.venue.price.tier,response.venue.price.message,response.venue.price.currency,response.venue.likes.count,response.venue.likes.groups,response.venue.likes.summary,response.venue.dislike,response.venue.ok,response.venue.rating,response.venue.ratingColor,response.venue.ratingSignals,response.venue.allowMenuUrlEdit,response.venue.beenHere.count,response.venue.beenHere.unconfirmedCount,response.venue.beenHere.marked,response.venue.beenHere.lastCheckinExpiredAt,response.venue.specials.count,response.venue.specials.items,response.venue.photos.count,response.venue.photos.groups,response.venue.venuePage.id,response.venue.reasons.count,response.venue.reasons.items,response.venue.description,response.venue.storeId,response.venue.page.user.firstName,response.venue.page.user.countryCode,response.venue.page.user.type,response.venue.page.user.venue.id,response.venue.page.user.tips.count,response.venue.page.user.lists.groups,response.venue.page.user.bio,response.venue.hereNow.count,response.venue.hereNow.summary,response.venue.hereNow.groups,response.venue.createdAt,response.venue.tips.count,response.venue.tips.groups,response.venue.shortUrl,response.venue.timeZone,response.venue.listed.count,response.venue.listed.groups,response.venue.hours.status,response.venue.hours.richStatus.entities,response.venue.hours.richStatus.text,response.venue.hours.isOpen,response.venue.hours.isLocalHoliday,response.venue.hours.dayData,response.venue.hours.timeframes,response.venue.popular.isOpen,response.venue.popular.isLocalHoliday,response.venue.popular.timeframes,response.venue.seasonalHours,response.venue.defaultHours.status,response.venue.defaultHours.richStatus.entities,response.venue.defaultHours.richStatus.text,response.venue.defaultHours.isOpen,response.venue.defaultHours.isLocalHoliday,response.venue.defaultHours.dayData,response.venue.defaultHours.timeframes,response.venue.pageUpdates.count,response.venue.pageUpdates.items,response.venue.inbox.count,response.venue.inbox.items,response.venue.attributes.groups,response.venue.bestPhoto.id,response.venue.bestPhoto.createdAt,response.venue.bestPhoto.source.name,response.venue.bestPhoto.source.url,response.venue.bestPhoto.prefix,response.venue.bestPhoto.suffix,response.venue.bestPhoto.width,response.venue.bestPhoto.height,response.venue.bestPhoto.visibility,response.venue.colors.highlightColor.photoId,response.venue.colors.highlightColor.value,response.venue.colors.highlightTextColor.photoId,response.venue.colors.highlightTextColor.value,response.venue.colors.algoVersion,response.venue.hasMenu,response.venue.delivery.id,response.venue.delivery.url,response.venue.delivery.provider.name,response.venue.delivery.provider.icon.prefix,response.venue.delivery.provider.icon.sizes,response.venue.delivery.provider.icon.name,response.venue.menu.type,response.venue.menu.label,response.venue.menu.anchor,response.venue.menu.url,response.venue.menu.mobileUrl,response.venue.parent.id,response.venue.parent.name,response.venue.parent.location.lat,response.venue.parent.location.lng,response.venue.parent.location.labeledLatLngs,response.venue.parent.location.cc,response.venue.parent.location.city,response.venue.parent.location.state,response.venue.parent.location.country,response

### Create and format data frame with only the data about the breweries that we need.

In [104]:
csb = pd.DataFrame(csb)

csb['Name'] = score['response.venue.name']
csb['id'] = score['response.venue.id']
csb['City'] = score['response.venue.location.city']
csb['Zip'] = score['response.venue.location.postalCode']
csb['LAT'] = score['response.venue.location.lat']
csb['LNG'] = score['response.venue.location.lng']
csb['Rating'] = score['response.venue.rating']
csb['Rating_Color'] = score['response.venue.ratingColor']

csbonly = csb.reset_index(drop=True)
csbonly

,Name,id,City,Zip,LAT,LNG,Rating,Rating_Color
0,Red Leg Brewing Company,512cfb53e4b01674115e19b8,Colorado Springs,80907,38.898057,-104.841728,8.3,73CF42
1,Cerberus Brewing Company,57c878a1498e9ad24cf1158d,Colorado Springs,80905,38.833003,-104.837443,9.3,00B551
2,Storybook Brewing,54750b26498e21a5eda35f5d,Colorado Springs,80907,38.877701,-104.812470,8.9,73CF42
3,Phantom Canyon Brewing Company,4ab97703f964a5207e7f20e3,Colorado Springs,80903,38.834215,-104.825177,8.8,73CF42
4,Bristol Brewing Company,4af49ef4f964a5206af421e3,Colorado Springs,80905,38.811236,-104.827392,8.6,73CF42
5,Trinity Brewing Company,4b118eadf964a5203c7f23e3,Colorado Springs,80907,38.897386,-104.854334,8.1,73CF42
6,Nano 108 Brewing Co,5274485711d283dea3d0fa0a,Colorado Springs,80915,38.866652,-104.717830,8.1,73CF42
7,Smiling Toad Brewing,50e374bce4b0a6d1deb767d4,Colorado Springs,80905,38.806723,-104.838538,8.1,73CF42
8,BJ's Restaurant & Brewhouse,4c731e737121a1cdd58464d1,Colorado Springs,80918,38.903441,-104.817887,8.2,73CF42
9,Colorado Mountain Brewery at the Roundhouse,504998e4e8891a6a2ccf22fc,Colorado Springs,80904,38.840201,-104.859583,7.9,C5DE35


### Map all the Colorado Springs Breweries, used the Font Awesome Beer Icon for fun.

In [203]:
# create map of Colorado Springs using latitude and longitude values
map_csb = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for i in range(len(csbonly)):
    label = 'Brewery-{}, Rating-{}'.format(csbonly['Name'][i],csbonly['Rating'][i])
    label = folium.Popup(label, parse_html=True)
    #folium.CircleMarker(
        #[CS['LAT'][i], CS['LNG'][i]],
        #radius=5,
        #popup=label,
        #color='blue',
        #fill=True,
        #fill_color='#3186cc',
        #fill_opacity=0.7,
        #parse_html=False).add_to(map_csprings)
    folium.Marker(
        [csbonly['LAT'][i], csbonly['LNG'][i]],
        popup=label,
        icon=folium.Icon(color='#FFD700',
        icon_color='#FFD700',
            icon='beer',
            prefix='fa'),
            ).add_to(map_csb)
    
map_csb

### Create dataframes for number of breweries in each zip, average rating of the breweries in each zip, and combine with base Colorado Springs zipcode list.

In [164]:
csb_rating_mean = csbonly.groupby('Zip').mean()

csb_rating_mean.columns = ['LAT', 'LNG', 'Avg_Rating']


In [165]:
csb_by_zip = csbonly.groupby('Zip').count()
csb_by_zip.drop(csb_by_zip.columns[[1,2,3,4,5,6]], axis = 1, inplace = True)
csb_by_zip.columns = ['Brewery_Count']


In [166]:
brewery_info = pd.merge(csb_by_zip,csb_rating_mean,on = 'Zip',how = 'inner')
brewery_info.drop(brewery_info.columns[[1,2]], axis = 1, inplace = True)
brewery_info

,Brewery_Count,Avg_Rating
Zip,,
80903,1,8.800000
80904,2,7.750000
80905,3,8.666667
80907,4,8.225000
80908,1,7.800000
80915,1,8.100000
80918,2,7.900000


### Dataframe with all zip, lat, long, brewery count and average rating.

In [214]:
CS_onehot = pd.merge(CS,brewery_info,on = 'Zip', how='left')
CS_onehot['Brewery_Count'] = CS_onehot['Brewery_Count'].fillna(0)
CS_onehot['Avg_Rating'] = CS_onehot['Avg_Rating'].fillna(0)
CS_onehot.drop(CS_onehot.columns[[1,2,5]], axis = 1, inplace = True)
CS_onehot

,Zip,LAT,LNG,Brewery_Count,Avg_Rating
0,80903,38.831526,-104.815062,1.0,8.800000
1,80904,38.860829,-104.876163,2.0,7.750000
2,80905,38.818910,-104.838348,3.0,8.666667
3,80906,38.752594,-104.878211,0.0,0.000000
4,80907,38.878231,-104.827015,4.0,8.225000
5,80908,39.047585,-104.690540,1.0,7.800000
6,80909,38.853475,-104.775218,0.0,0.000000
7,80910,38.812066,-104.774580,0.0,0.000000
8,80911,38.752916,-104.723477,0.0,0.000000
9,80913,38.687462,-104.748767,0.0,0.000000


### Create Clusters and add Cluster labels to dataframe.

In [215]:
# set number of clusters
kclusters = 4

CS_clustering = CS_onehot.drop('Zip', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(CS_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 3, 2, 0, 2, 1, 0, 0, 0, 0])

In [216]:
# add clustering labels
CS_onehot.insert(0, 'Cluster Labels', kmeans.labels_)

CS_onehot 

,Cluster Labels,Zip,LAT,LNG,Brewery_Count,Avg_Rating
0,1,80903,38.831526,-104.815062,1.0,8.800000
1,3,80904,38.860829,-104.876163,2.0,7.750000
2,2,80905,38.818910,-104.838348,3.0,8.666667
3,0,80906,38.752594,-104.878211,0.0,0.000000
4,2,80907,38.878231,-104.827015,4.0,8.225000
5,1,80908,39.047585,-104.690540,1.0,7.800000
6,0,80909,38.853475,-104.775218,0.0,0.000000
7,0,80910,38.812066,-104.774580,0.0,0.000000
8,0,80911,38.752916,-104.723477,0.0,0.000000
9,0,80913,38.687462,-104.748767,0.0,0.000000


### Create map with pop up of each zip code detailing the number of breweries and average rating, color coded by cluster.

In [217]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for i in range(len(CS_onehot)):
    label = folium.Popup(str(CS_onehot['Zip'][i]) + ' Cluster ' + str(CS_onehot['Cluster Labels'][i]) + 'Breweries ' + str(CS_onehot['Brewery_Count'][i]) + 'Avg. Rating '+str(CS_onehot['Avg_Rating'][i]), parse_html=True)
    folium.CircleMarker(
        [CS_onehot['LAT'][i], CS_onehot['LNG'][i]],
        radius=5,
        popup=label,
        color=rainbow[CS_onehot['Cluster Labels'][i]-2],
        fill=True,
        fill_color=rainbow[CS_onehot['Cluster Labels'][i]-2],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters